Levenshtein con palabras en lugar de carácteres con los documentos que ya recuperamos para regresar los subtitulos más parecidos a la query

In [1]:
import numpy as np
from collections import defaultdict
import pickle
from tqdm import tqdm
import pandas as pd

In [2]:
def flatten(lst):
    """
    lst: lista de lista
    Regresa una lista con una dimension menos que la pasada 
    """
    return [item for sub in lst for item in sub]

In [3]:
def findIx(video, ix):
    """
    video: video en el que se buscara
    ix: indice de la palabra que buscamos
    regresa el subtitulo y su indice de la palabra buscada
    """
    subtitles = video["subtitles"]
    r = -1
    while ix>0:
        r += 1
        ix -= len(subtitles[r]["text"])
    return (r, subtitles[r])

In [4]:
with open("../pkl/clean_videos.pkl", "rb") as f:
    videos = pickle.load(f)
videos[0][0]

{'id': 'L9YhoRatRzE',
 'original_title': 'Siempre Fui Yo | Adelanto | Disney+',
 'subtitles': [{'start': '0.13',
   'dur': '3.77',
   'text': ['tu', 'papá', 'tuvo', 'un', 'accidente']},
  {'start': '12.5', 'dur': '5.939', 'text': ['te', 'recuerdo', 'que', 'está']},
  {'start': '15.59', 'dur': '5.339', 'text': ['aquí']},
  {'start': '18.439',
   'dur': '6.361',
   'text': ['estaba', 'como', 'rabioso', 'con']},
  {'start': '20.929',
   'dur': '7.65',
   'text': ['especial', 'con', 'lucas', 'martín']},
  {'start': '24.8',
   'dur': '3.779',
   'text': ['necesito', 'saber', 'qué', 'fue', 'lo', 'que', 'pasó']},
  {'start': '29.42', 'dur': '2.479', 'text': ['aplausos']}]}

In [5]:
documents = defaultdict(list)
def merge_videos(corpus):
    '''
    Mezcla el corpus de cada video en una entrada del
    diccionario movies en un solo corpus, 
    guarda los diálogos por línea tokenizados y limpios

    Args:
        col (dic): Diccionario con los diálogos
    '''
    corpus_clean = []
    #Iteramos sobre los canales
    for chanel in tqdm(corpus):
        for video in chanel:
            #Cada texto se guarda por oraciones
            if 'subtitles' in video:
                corpus_clean += [s['text'] for s in video['subtitles']]
                for s in video['subtitles']:
                    documents[video['id']] += s["text"]
    return corpus_clean

all_videos = merge_videos(videos)
all_videos[0]

100%|████████████████████████████████████████████| 9/9 [00:00<00:00, 25.63it/s]


['tu', 'papá', 'tuvo', 'un', 'accidente']

In [6]:
def getVideosDict():
    """
    regresa un diccionario donde el id de un video es su llave
    """
    return {v['id']:v for v in flatten(videos)}
videosDict = getVideosDict()

In [7]:
def printDistances(distances, token1Length, token2Length):
    """
    Función auxiliar para imprimir la matriz de las distancias de Levenshtein
    """
    for t1 in range(token1Length + 1):
        for t2 in range(token2Length + 1):
            print(int(distances[t1][t2]), end=" ")
        print()

In [8]:
def faux_levenshtein(q, d, verbose=False):
    """
    q: query a buscar
    d: string en la que buscamos
    verbose: si la función debe imprimir la matrix
    faux leveshtein ejecuta leveshtein ejecuta leveshtein con matrix, pero toda la columna 
    del espacio de busqueda es inicalizada con 0 lo que nos permite encontrar coincidencias
    como subcandena y comparadas desde el inico.
    Regresa la distancia minima de leveshtein entre nuestra query y una subcadena de d y el 
    indice de dicha subcadena.
    """
    distances = np.zeros((len(q) + 1, len(d) + 1))
    for t1 in range(len(q) + 1):
        distances[t1][0] = t1
    a = 0
    b = 0
    c = 0
    
    for t1 in range(1, len(q) + 1):
        for t2 in range(1, len(d) + 1):
            if (q[t1-1] == d[t2-1]):
                distances[t1][t2] = distances[t1 - 1][t2 - 1]
            else:
                a = distances[t1][t2 - 1]
                b = distances[t1 - 1][t2]
                c = distances[t1 - 1][t2 - 1]
                if (a <= b and a <= c):
                    distances[t1][t2] = a + 1
                elif (b <= a and b <= c):
                    distances[t1][t2] = b + 1
                else:
                    distances[t1][t2] = c + 1
    if(verbose):
        printDistances(distances, len(q), len(d))
    
    min_arg = np.argmin(distances[len(q)])
    return (distances[len(q)][min_arg], min_arg-len(q))

In [54]:
def search_faux_levenshtein(q, **kwargs):
    """
    q: arreglo de cadenas que queremos encontrar en el corpues
    regresa un dataframe ordenado por su destancia de faux levenshtein
    """
    use_tqdm = ('use_tqdm' in kwargs) and kwargs['use_tqdm'] 
    indexes = []
    minvals = []
    for (k,d) in tqdm(documents.items(), position=0):
        minval, mix =faux_levenshtein(q, d)
        indexes.append(mix)
        minvals.append(minval)
    df = pd.DataFrame(data={"id":documents.keys(), 'faux leveshtein':minvals, 'fl index':indexes})
    return df.sort_values(by=['faux leveshtein'])

In [55]:
def search(q):
    """
    q: cadena que queremos encontrar en el corpus
    regresa un generador que podemos iterar para ir encontrando los resultados de la busqueda
    """
    def iterator(top):
        for _,serie in top.iterrows():
            ix = serie['fl index']
            video = videosDict[serie["id"]]
            subtitle_id, subtitle = findIx(video, ix)
            next_sub = None
            if (subtitle_id+1) < len(video["subtitles"]):
                next_sub = video["subtitles"][subtitle_id+1]["text"]
            yield {
                "id":serie["id"], 
                "subtitle": subtitle, 
                "next_subtitle": next_sub,
                "subtitle_id":subtitle_id, 
                'faux leveshtein':serie['faux leveshtein'] 
            }
    q = q.split(" ")
    top = search_faux_levenshtein(q)
    return iterator(top)

In [56]:
r = search("grande y gordo")

100%|█████████████████████████████████████| 3278/3278 [00:14<00:00, 232.62it/s]


In [57]:
def top5(query):
    fl = search("grande y gordo")
    return [next(fl)["subtitle"]["text"] for i in range(5)]

In [58]:
with open("../pkl/test.pkl", "rb") as f:
    test_all = pickle.load(f)

test_all[3000:3010]

['sí',
 'españoles son muy hábiles porque',
 'quiero a entonces la cosa cambia porque',
 'qué trozo de pizza tripa de paté ketchup',
 'serie',
 'cuerpo no tiene un pelo de tonto',
 'cómo',
 'muerto para que no sospecharan de él',
 '¿Ven?',
 'esto es 31 minutos vamos a un adelanto']

In [66]:
resp = []
for ix, q in enumerate(test_all):
    if ix%10==0:
        pickle.dump(resp, open("../pkl/faux_levenshtein.pkl", "wb"))
        print(f"salvando {ix} elementos")
    resp.append(top5(q))

salvando 0 elementos


100%|█████████████████████████████████████| 3278/3278 [00:14<00:00, 226.19it/s]


salvando 10 elementos


100%|█████████████████████████████████████| 3278/3278 [00:14<00:00, 231.26it/s]


salvando 20 elementos


...

salvando 4610 elementos


100%|███████████████████████████████████| 3278/3278 [00:23<00:00, 142.49it/s]


salvando 4620 elementos


 27%|█████████▋                          | 877/3278 [00:06<00:18, 127.27it/s]


KeyboardInterrupt: 